# Прогнозирование стоимости автомобиля по характеристикам

## Задача - создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам. 

### Описание признаков
ляллялялял

## Import

In [1]:
from IPython.core.display import display, HTML

import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
from pandas import Series
import numpy as np
from numpy import mean
import json

from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.feature_selection import f_classif, mutual_info_classif, VarianceThreshold, SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MiniBatchKMeans
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import kneighbors_graph
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics.cluster import completeness_score, v_measure_score

from sklearn.metrics import silhouette_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

from scipy.stats import randint
import statsmodels.api as sm

from imblearn.pipeline import Pipeline

from math import log as log
import os

from itertools import combinations
from scipy.stats import ttest_ind

import re

In [2]:
import warnings; warnings.simplefilter('ignore')

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
RANDOM_SEED = 42

## Helpers

In [5]:
class OwnershipPeriod():
    def __init__(self):
        self.pattern = re.compile('((\d+) (года|год|лет)){0,1}( и ){0,1}((\d+) (месяц)){0,1}')
        
    def get_months_from_ownership_period(self, x):
        '''Функция для подсчета кол-ва месяцев владения автомобилем, осуществляется через 
        преобразование кол-ва лет и месяцев в кол-во месяцев.'''
        search_res = self.pattern.search(str(x))

        years = 0
        months = 0

        if search_res.group(2):
            years = int(search_res.group(2))
        if search_res.group(6):
            months = int(search_res.group(6))

        return years*12 + months

In [6]:
def profile_report(df):
    '''Функция для вызова профайл-отчета о датафрейме'''
    
    if df is None:
        raise TypeError(nameof(df))

    profile = df.profile_report(
        title='Car Price prediction',
        progress_bar=False,
        correlations={
            'pearson': {'calculate': True},
            'spearman': {'calculate': True},
            'kendall': {'calculate': True},
            'phi_k': {'calculate': True},
            'cramers': {'calculate': True},
        },
        interactions={
            'continuous': True,
            'targets': []
        },
        missing_diagrams={
            'heatmap': True,
            'dendrogram': True,
            'matrix': True
        },
        vars={
            'cat': {'n_obs': 10}
        }
    )

    return profile

In [7]:
def asfloat(x, zero_if_none=False):
    if x is None or x == '':
        if zero_if_none:
            return 0
        else:
            return None
    else:
        return float(x)    

In [34]:
def asdict(x):
    if type(x) != str:
        return None
    
    return eval(x.replace('true', 'True').replace('false', 'False'))

## Data

В рамках задания были предоставлены тестовые данные. Для получения обучающей выборки был произведен парсинг данных с сайта auto.ru. В версии ноутбука на github представлен код для парсинга. В рамках ноутбука на Kaggle загружается готовый датасет без указания кода для парсинга.

In [8]:
train = pd.read_csv('parsed_data.csv')
test = pd.read_csv('test.csv')

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52055 entries, 0 to 52054
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              52055 non-null  object 
 1   brand                 52055 non-null  object 
 2   car_url               52055 non-null  object 
 3   color                 52055 non-null  object 
 4   complectation_dict    10058 non-null  object 
 5   description           50321 non-null  object 
 6   engineDisplacement    52055 non-null  object 
 7   enginePower           52055 non-null  object 
 8   equipment_dict        52055 non-null  object 
 9   fuelType              52055 non-null  object 
 10  image                 0 non-null      float64
 11  mileage               52055 non-null  int64  
 12  modelDate             52055 non-null  int64  
 13  model_info            52055 non-null  object 
 14  model_name            52055 non-null  object 
 15  name               

In [10]:
train.isna().sum()

bodyType                    0
brand                       0
car_url                     0
color                       0
complectation_dict      41997
description              1734
engineDisplacement          0
enginePower                 0
equipment_dict              0
fuelType                    0
image                   52055
mileage                     0
modelDate                   0
model_info                  0
model_name                  0
name                        0
numberOfDoors               0
parsing_unixtime            0
priceCurrency               0
productionDate              0
sell_id                     0
super_gen                   0
vehicleConfiguration        0
vehicleTransmission         0
vendor                      0
Владельцы                   0
Владение                35436
ПТС                         2
Привод                      0
Руль                        0
Состояние                   0
Таможня                     0
price                       0
dtype: int

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [12]:
test.isna().sum()

bodyType                    0
brand                       0
car_url                     0
color                       0
complectation_dict      28268
description                 0
engineDisplacement          0
enginePower                 0
equipment_dict           9996
fuelType                    0
image                       0
mileage                     0
modelDate                   0
model_info                  0
model_name                  0
name                        0
numberOfDoors               0
parsing_unixtime            0
priceCurrency               0
productionDate              0
sell_id                     0
super_gen                   0
vehicleConfiguration        0
vehicleTransmission         0
vendor                      0
Владельцы                   0
Владение                22691
ПТС                         1
Привод                      0
Руль                        0
Состояние                   0
Таможня                     0
dtype: int64

## Prepping data

Для корректной работы над данными и построения модели, необходимо провести первоначальную обработку имеющихся данных, чтобы наблюдения в тестовой и обучающей выборке были отражены одинаковым образом.

Посмотрим, есть ли у нас дубликаты в датасетах.

In [13]:
train['sell_id'].duplicated().sum()

0

In [14]:
test['sell_id'].duplicated().sum()

0

Объединим датасеты для дальнейшей обработки.

In [15]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест

data = test.append(train, sort=False).reset_index(drop=True) # объединяем

Проверим на дубликаты. Нет ли у нас в обучающей выборке наблюдений из тестовой, - иначе модель может работать не корректно.

In [16]:
data['sell_id'].duplicated(keep=False).sum()

3544

In [17]:
data[data.sell_id.duplicated(keep=False)].sort_values(['sell_id']).head(6)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,sample,price
60936,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Mercedes-benz 560 sel произведен известной фир...,5.5 LTR,279 N12,"{'cruise-control': True, 'tinted-glass': True,...",бензин,NaN,92000,1985,"{'code': 'S_KLASSE', 'name': 'S-Класс', 'ru_na...",S_KLASSE,560 5.6 AT (279 л.с.),4,1620552949,RUB,1986,2665,"{'id': '20470810', 'name': '560', 'nameplate':...",SEDAN AUTOMATIC 5.5 LTR,автоматическая,EUROPEAN,1 владелец,15 лет и 2 месяца,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1,3500000.0
17153,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Mercedes-benz 560 sel произведен известной фир...,5.6 LTR,279 N12,"{""cruise-control"":true,""tinted-glass"":true,""ro...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,92000,1985,"{""code"":""S_KLASSE"",""name"":""S-Класс"",""ru_name"":...",S_KLASSE,560 5.6 AT (279 л.с.),4,1603246796,RUB,1986,2665,"{""id"":""20470810"",""name"":""560"",""nameplate"":""560...",SEDAN AUTOMATIC 5.6,автоматическая,EUROPEAN,1 владелец,14 лет и 7 месяцев,Оригинал,задний,Левый,Не требует ремонта,Растаможен,0,NaN
36751,универсал 5 дв.,AUDI,https://auto.ru/cars/used/sale/audi/a4/4894588...,серебристый,NaN,"Автомобиль в идеальном состоянии. Ездит мало, ...",1.8 LTR,170 N12,{},бензин,NaN,77000,2011,"{'code': 'A4', 'name': 'A4', 'ru_name': 'А4', ...",A4,1.8 CVT (170 л.с.),5,1620552938,RUB,2013,48945886,"{'id': '7765817', 'displacement': 1798, 'engin...",WAGON_5_DOORS VARIATOR 1.8 LTR,вариатор,EUROPEAN,1 владелец,7 лет и 8 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1,1290000.0
3594,универсал 5 дв.,AUDI,https://auto.ru/cars/used/sale/audi/a4/4894588...,серый,NaN,Автомобиль в идеальном состоянии. Продаю потом...,1.8 LTR,170 N12,NaN,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,77000,2011,"{""code"":""A4"",""name"":""A4"",""ru_name"":""А4"",""morph...",A4,1.8 CVT (170 л.с.),5,1603126369,RUB,2013,48945886,"{""id"":""7765817"",""displacement"":1798,""engine_ty...",WAGON_5_DOORS VARIATOR 1.8,вариатор,EUROPEAN,1 владелец,7 лет,Оригинал,передний,Левый,Не требует ремонта,Растаможен,0,NaN
28035,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/outl...,серебристый,NaN,"Не битый, не крашенный (ни одна деталь), с по...",2.0 LTR,146 N12,"{""asr"":true,""tinted-glass"":true,""esp"":true,""is...",бензин,https://autoru.naydex.net/mcQh17801/f1b5751dCi...,270000,2012,"{""code"":""OUTLANDER"",""name"":""Outlander"",""ru_nam...",OUTLANDER,2.0 CVT (146 л.с.) 4WD,5,1603264934,RUB,2012,1003205882,"{""id"":""8352278"",""displacement"":1998,""engine_ty...",ALLROAD_5_DOORS VARIATOR 2.0,вариатор,JAPANESE,1 владелец,7 лет и 7 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,0,NaN
65437,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/outl...,серый,NaN,"Не битый, не крашенный (ни одна деталь), с по...",2.0 LTR,146 N12,"{'asr': True, 'tinted-glass': True, 'esp': Tru...",бензин,NaN,270000,2012,"{'code': 'OUTLANDER', 'name': 'Outlander', 'ru...",OUTLANDER,2.0 CVT (146 л.с.) 4WD,5,1620552951,RUB,2012,1003205882,"{'id': '8352278', 'displacement': 1998, 'engin...",ALLROAD_5_DOORS VARIATOR 2.0 LTR,вариатор,JAPANESE,1 владелец,8 лет и 3 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1,980000.0


Такие наблюдения у нас есть. Удалим дубликаты пришедшие из обучающей выборки.

In [18]:
data = data.drop_duplicates(subset=['sell_id'], keep='first')

In [19]:
# Переимунуем колонки
data.rename(columns={
    'Владельцы': 'owners_number', 
    'Владение': 'ownership_period',
    'ПТС': 'pts', 
    'Привод': 'gear_type',
    'Руль': 'steering_wheel', 
    'Состояние': 'state', 
    'Таможня': 'custom_cleared'
}, inplace=True)

In [20]:
# Преобразуем значения в колонке steering_wheel
data['steering_wheel'] = data['steering_wheel'].apply(lambda x: 1 if x == 'Левый' else 0).astype('int8')

In [21]:
# Преобразуем значения в колонке pts
data['pts'] = data['pts'].apply(lambda x: 1 if x == 'Оригинал' else 0).astype('int8')

In [22]:
# Преобразуем значения в колонке bodyType
data['bodyType'] = data['bodyType'].apply(lambda x: x.split(' ')[0])

In [23]:
# Преобразуем значения в колонке engineDisplacement
# заменим пустые значения на 0, так как, судя по данным, они все относятся к электрокарам
data['engineDisplacement'] = data['engineDisplacement'].apply(lambda x: asfloat(x[:-4], zero_if_none=True)).astype('float32')

In [24]:
# Преобразуем значения в колонке enginePower
data['enginePower'] = data['enginePower'].apply(lambda x: int(x[:-4])).astype('int32')

In [25]:
# Преобразуем значения в колонке owners_number
data['owners_number'] = data['owners_number'].apply(lambda x: int(x[0])).astype('int32')

In [26]:
# Преобразуем значения в колонке ownership_period - переведем все в кол-во месяцев
ownership_processor = OwnershipPeriod()

data['ownership_period'] = data['ownership_period'].apply(ownership_processor.get_months_from_ownership_period).astype('int32')

Далее преобразуем строковые величины в виде словарей в словари. Далее, возможно, возьмем оттуда информацию для улучшения точности модели. Из признака super_gen сразу возьмем значения расхода топлива и разгона.

In [27]:
# super_gen
data['super_gen'] = data['super_gen'].apply(lambda x: eval(x))

data['fuel_rate'] = data['super_gen'].apply(lambda x: x['fuel_rate'] if 'fuel_rate' in x.keys() else 0)
data['acceleration'] = data['super_gen'].apply(lambda x: x['acceleration'] if 'acceleration' in x.keys() else 0)

In [31]:
# equipment_dict
data['equipment_dict'] = data['equipment_dict'].apply(asdict)

In [35]:
# complectation_dict
data['complectation_dict'] = data['complectation_dict'].apply(asdict)

Удалим признаки, которые не несут для нас необходимой информации или дублируют то, что у нас уже представлено в других признаках.

In [37]:
data.drop(columns=[
    'name', 
    'model_info', 
    'image', 
    'vehicleConfiguration', 
    'car_url', 
    'parsing_unixtime',
    'priceCurrency', 
    'super_gen', 
    'state', 
    'custom_cleared'
], inplace=True)

In [38]:
# Подготовим данные для осмотра в HTML формате, ссылка на данные будет ниже

data_profile = profile_report(data)
data_profile.to_file('output.html')

# !!! Я дошел до эотго места !!!

## Data Inspection

In [ ]:
# По желанию, профайл можно посмотреть внутри ноутбука.

# pandas_profiling.ProfileReport(data)

#### Ссылка на профайл **[the dataset profile report](./output.html)**.

### На основании профайлинга можем видеть следующее:

1. 

Маленький эксперимент

In [ ]:
le = LabelEncoder()

data['bodyType'] = le.fit_transform(data['bodyType'])
data['brand'] = le.fit_transform(data['brand'])
data['color'] = le.fit_transform(data['color'])
data['fuelType'] = le.fit_transform(data['fuelType'])
data['model_name'] = le.fit_transform(data['model_name'])
data['vehicleTransmission'] = le.fit_transform(data['vehicleTransmission'])
data['vendor'] = le.fit_transform(data['vendor'])
data['gear_type'] = le.fit_transform(data['gear_type'])

In [ ]:
data.info()

In [ ]:
train_data = data.query('sample == 1').drop(['sample', 'complectation_dict', 'description', 'equipment_dict'], axis=1)

In [ ]:
train_data = train_data.dropna()

In [ ]:
X = train_data.drop(['price'], axis=1)
y = train_data['price']

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mean_absolute_percentage_error(y_test, predict))*100:0.2f}%")